In [ ]:
import os
import httpx
import json
from crewai import Agent, Task, Crew
from langchain.agents import tool
from langchain_community.llms import Ollama

# Directory for your domain-specific documents (assumed to be .txt files)
DOCS_DIR = "docs"

# Load documents from the 'docs' directory
def load_documents():
    """Loads all .txt files from the 'docs' directory and returns a list of (filename, content) tuples."""
    if not os.path.exists(DOCS_DIR):
        return []
    documents = []
    for filename in os.listdir(DOCS_DIR):
        if filename.endswith(".txt"):
            with open(os.path.join(DOCS_DIR, filename), "r", encoding="utf-8") as f:
                content = f.read()
                documents.append((filename, content))
    return documents

DOCUMENTS = load_documents()

@tool
def google_search(query: str) -> str:
    """Searches Google using Serper API for web results based on the query.

    Args:
        query (str): The search query.

    Returns:
        str: A summary of Google search results from Serper or a message if the search fails.
    """
    url = "https://google.serper.dev/search"
    headers = {
        "X-API-KEY": "ebda9508d11e4cc8c5091494a76b783fda7fe6da",
        "Content-Type": "application/json"
    }
    response = httpx.post(url, json={"q": query}, headers=headers)
    if response.status_code == 200:
        data = response.json()
        results = []
        for result in data.get("organic", []):
            title = result.get("title", "No title")
            snippet = result.get("snippet", "")
            link = result.get("link", "")
            results.append(f"{title}: {snippet} ({link})")
            if len(results) >= 3:
                break
        return "\n".join(results) if results else "No results found using Google Serper."
    return "Error performing Google Serper search."

@tool
def search_docs(query: str) -> str:
    """Searches local documents for relevant information based on the query.
    
    Args:
        query (str): The search query.
    
    Returns:
        str: Snippets from documents matching the query or a message if no documents are found.
    """
    if not DOCUMENTS:
        return "No documents found."
    query_words = query.lower().split()
    results = []
    for filename, content in DOCUMENTS:
        lines = content.split('\n')
        for line in lines:
            if any(word in line.lower() for word in query_words):
                results.append(f"From {filename}: {line[:200]}")
                if len(results) >= 3:
                    break
        if len(results) >= 3:
            break
    return "\n".join(results) if results else "No relevant information found in documents."

@tool
def wikipedia(query: str) -> str:
    """Searches Persian Wikipedia for a summary based on the query.
    
    Args:
        query (str): The search query.
    
    Returns:
        str: The snippet from the first Wikipedia search result or a message if no results are found.
    """
    response = httpx.get("https://fa.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json",
        "utf8": 1
    })
    results = response.json().get("query", {}).get("search", [])
    if results:
        return results[0].get("snippet", "اطلاعاتی یافت نشد.")
    return "اطلاعاتی یافت نشد در ویکی‌پدیا."

@tool
def duckduckgo(query: str) -> str:
    """Searches DuckDuckGo for web results based on the query.
    
    Args:
        query (str): The search query.
    
    Returns:
        str: A placeholder message with search results or a message if the search fails.
    """
    response = httpx.get("https://duckduckgo.com/html/", params={"q": query})
    if response.status_code == 200:
        return "نتایج جستجوی DuckDuckGo: [صفحه 1, صفحه 2, صفحه 3]"
    return "نتایجی از DuckDuckGo یافت نشد."

# Set up the language model (using Llama3.2 for now)
llm = Ollama(model="llama3.2")

# Create the agent with CrewAI prioritizing Google Serper search first
agent = Agent(
    role="researcher",
    goal="Answer the user's question by primarily using Google search results from the Serper API, then using local documents, Persian Wikipedia, and DuckDuckGo if needed.",
    backstory="You are an AI assistant designed to efficiently gather and synthesize information from multiple sources, prioritizing Google search results from the Serper API.",
    tools=[google_search, search_docs, wikipedia, duckduckgo],
    llm=llm,
    verbose=True
)

def query(question):
    """Runs the query through the agent and saves the result.
    
    Args:
        question (str): The user's question.
    """
    task = Task(
        description=question,
        expected_output="A detailed answer to the user's question.",
        agent=agent
    )
    crew = Crew(
        agents=[agent],
        tasks=[task],
        verbose=2
    )
    result = crew.kickoff()
    print(result)
    save_to_file(str(result))

def save_to_file(text, path="C:/Users/ASUS/Downloads/Month 2/agents.txt"):
    """Saves the result to a file.
    
    Args:
        text (str): The text to save.
        path (str): The file path to save the text.
    """
    with open(path, 'a', encoding='utf-8') as f:
        f.write(text + "\n" + "="*50 + "\n")

if __name__ == "__main__":
    # Example queries to test the agent
    q1 = "شاه عباس صفوی آدم خوبی بوده؟ چرا؟"
    q2 = "وقتی چراغ DNS مودم قطع میشه به چه معنیه؟"
    query(q1)
    query(q2)